# Notebook to compute the DHW and DCW indices 

In [ ]:
# load libraries
import pandas as pd
import numpy as np

In [ ]:
# Import file with processed data in .h5 format and open it as a dataframe 'df'
## NOTE: This data can be avoided if a dtaframe is already created ##
with pd.HDFStore('df_SST_CRW.h5','r') as input:
    df_SST=input.get('df_SST_CRW')

In [ ]:
# inspect data
df_SST.head()

In [ ]:
def arrange_df (df):
    'This function reindex, sort_values, interpolate NaN´s and append the average of the week and average of month'
    df = df.reindex(columns=['site_name','year','month','week','days','analysed_sst']) # Re-order by column starting by 'site_name'
    df = df.sort_values(by=['site_name','year','month','week','days']) # Re-order in ascendent mode
    df=df.interpolate() # interpolate missing values averaging the nearest values.
    df['sst_week'] = df.groupby(['site_name','year','month','week'])['analysed_sst'].transform('mean') # add column of week value
    df['sst_month'] = df.groupby(['site_name','year','month'])['analysed_sst'].transform('mean')
    df['sst_clim'] = df[(df.year<=2005)].groupby(['site_name','month'])['analysed_sst'].transform('mean') # compute climatology from 1985-2005
    df['MMmin'] = df.groupby(['site_name'])['sst_clim'].transform('min') #This gives the minimun climatology SST 
    df['MMMax'] = df.groupby(['site_name'])['sst_clim'].transform('max') #This gives the maximum climatology SST
    return df

In [ ]:
df_SST = arrange_df(df_SST) # apply function
df_SST = df_SST.drop(columns=['sst_clim'])  # drop climatology 

# HotSpots
## Use the MMMAx and MMmin to compute the HotSpot and ColdSpot respectively

In [ ]:
def get_HotSpot (df):
    HotSpot = df_SST.groupby(['site_name','year','week'])['sst_week'].mean()-df_SST.groupby(['site_name','year','week'])['MMMax'].mean()
    return pd.DataFrame(HotSpot)

In [ ]:
HotSpot = get_HotSpot(df_SST)

## Only HotSpot positives and grater than 1 are going to be sum

In [ ]:
# Dismiss values lower than 1 
HotSpot = round(HotSpot, 3)
hit = HotSpot < 1
HotSpot[hit] = 0

# Degree Heating Weeks

In [ ]:
def DHW (df):
    ' function to compute the DHW index; it sum only positive values 1°C > MMMax in a rolling 12 weeks window'
    df = round(df.rolling(center=False,window=12).sum(),3) # Sum the values in the windows of "two" weeks
    df['site_name'] = df.index.get_level_values(level=0) # assign new columns based on hiden levels
    df['year'] = df.index.get_level_values(level=1)
    df['week'] = df.index.get_level_values(level=2)
    df.columns = ['DHW','site_name','year','week'] # Re-name the columns
    df = df.reset_index(level=(0,1,2), drop=True) # Drop the hiden levels
    df = df.reindex(columns = ['site_name','year','week','DHW']) # Re-order the columns 
    return df

In [ ]:
DHW = DHW(HotSpot)

# ColdSpots 

In [ ]:
# The cold spot is the difference of the month value and the minimum climatology
def get_ColdSpot (df):
    ColdSpot = df_SST.groupby(['site_name','year','week'])['sst_week'].mean()-df_SST.groupby(['site_name','year','week'])['MMmin'].mean()
    return pd.DataFrame(ColdSpot)

In [ ]:
ColdSpot = get_ColdSpot(df_SST)

## Only ColdSpots negative and lower than 1°C are going to be sum

In [ ]:
ColdSpot = round(ColdSpot, 2)
hit = ColdSpot >= 0
ColdSpot[hit] = 0

# Degree Cooling Weeeks

In [ ]:
def DCW (df):
    ' function to compute the DCW index; it sum only values -1°C < MMmin in a rolling 12 weeks window'
    df = round(df.rolling(center=False,window=3).sum(),3) # Sum the values in the windows of three months
    df['site_name'] = df.index.get_level_values(level=0) # assign new columns based on hiden levels
    df['year'] = df.index.get_level_values(level=1)
    df['week'] = df.index.get_level_values(level=2)
    df.columns = ['DCW','site_name','year','week'] # Rename the columns
    df = df.reset_index(level=(0,1,2), drop=True) # Drop the hiden levels
    df = df.reindex(columns = ['site_name','year','week','DCW']) # Re-order the columns 
    return df

In [ ]:
DCW = DCW(ColdSpot)

In [ ]:
table.to_csv('table_DCM.csv')